# Video "What is TensorFlow"

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0-dev20190914


In [2]:
a = tf.constant(tf.ones((2,2)))
b = tf.Variable(tf.zeros((2,2)))

In [3]:
a

<tf.Tensor: id=2, shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [4]:
b

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0., 0.],
       [0., 0.]], dtype=float32)>

In [5]:
b = tf.ones((2, 2))

In [6]:
c = a @ b
c

<tf.Tensor: id=18, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

## First model in TensorFlow 2.0

In [7]:
x = tf.Variable(3,name = "x", trainable=True,dtype=tf.float32)
x

<tf.Variable 'x:0' shape=() dtype=float32, numpy=3.0>

In [8]:
x.numpy()

3.0

In [9]:
def y_fun():
    y = (x - 5) ** 2
    return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.3)
for i in range(10):
    step = optimizer.minimize(y_fun, var_list=[x])
    print(x.numpy(),y_fun().numpy())

4.2 0.6400003
4.68 0.10240011
4.8719997 0.016384067
4.9488 0.0026214311
4.97952 0.00041943678
4.991808 6.710989e-05
4.996723 1.0737582e-05
4.998689 1.7182631e-06
4.9994755 2.7512215e-07
4.99979 4.4019544e-08


### Simple optimization (with tf.Print)

In [10]:
x = tf.Variable(3,name = "x", trainable=True,dtype=tf.float32)

def y_fun():
    y = (x - 5) ** 2
    #tf.print(y, x,"x, f:")
    tf.print("x : ", x,", y: " , y)
    return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.3)
for i in range(10):
    step = optimizer.minimize(y_fun, var_list=[x])

x :  3 , y:  4
x :  4.2 , y:  0.640000284
x :  4.68 , y:  0.102400109
x :  4.87199974 , y:  0.016384067
x :  4.9488 , y:  0.00262143114
x :  4.97952 , y:  0.000419436779
x :  4.99180794 , y:  6.71098896e-05
x :  4.99672318 , y:  1.07375818e-05
x :  4.99868917 , y:  1.71826309e-06
x :  4.99947548 , y:  2.75122147e-07


### Simple optimization (with TensorBoard logging)

In [11]:
# ! pip install tf-nightly-2.0-preview tf_estimator_nightly

In [12]:
%load_ext tensorboard

In [13]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [14]:
directory = 'logs/simple1'
summary_writer = tf.summary.create_file_writer(directory)

In [15]:
x = tf.Variable(3,name = "x", trainable=True,dtype=tf.float32)

def y_fun():
    y = (x - 5) ** 2
    #tf.print(y, x,"x, f:")
    #tf.print("x : ", x," y: " , y)
    return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.3)
for i in range(10):
    step = optimizer.minimize(y_fun, var_list=[x])
    
    with summary_writer.as_default():
        tf.summary.scalar('x', x, step=i)
        tf.summary.scalar('y', y_fun(), step=i)

In [16]:
%tensorboard --logdir logs/simple1

Reusing TensorBoard on port 6006 (pid 24983), started 0:02:56 ago. (Use '!kill 24983' to kill it.)